In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path

# Load all regional data files
regions = ['midwest', 'northeast', 'northwest', 'south']
data_dir = Path('future_data')

# Load and combine all regional data
all_data = []
for region in regions:
    df = pd.read_csv(data_dir / f'{region}_futures_merged.csv')
    df['region'] = region
    all_data.append(df)

# Combine all regions
combined_df = pd.concat(all_data, ignore_index=True)

# Calculate average precipitation across all regions for each year
avg_precip = combined_df.groupby('year').agg({
    'low_emissions_pr': 'mean',
    'high_emissions_pr': 'mean'
}).reset_index()

# Filter for years 2025-2100
avg_precip = avg_precip[(avg_precip['year'] >= 2025) & (avg_precip['year'] <= 2100)]

# Unit conversion factor
CONVERSION_FACTOR = 1000 

avg_precip['low_emissions_pr_mm'] = avg_precip['low_emissions_pr'] * CONVERSION_FACTOR
avg_precip['high_emissions_pr_mm'] = avg_precip['high_emissions_pr'] * CONVERSION_FACTOR

# Create the line chart
fig = go.Figure()

# Add SSP1-2.6 (low emission) line
fig.add_trace(go.Scatter(
    x=avg_precip['year'],
    y=avg_precip['low_emissions_pr_mm'],
    mode='lines',
    name='SSP1-2.6 (Low Emission)',
    line=dict(color='#2E86AB', width=3),
    hovertemplate='<b>SSP1-2.6</b><br>' +
                  'Year: %{x}<br>' +
                  'Precipitation: %{y:.2f} mm/year<extra></extra>'
))

# Add SSP5-8.5 (high emission) line
fig.add_trace(go.Scatter(
    x=avg_precip['year'],
    y=avg_precip['high_emissions_pr_mm'],
    mode='lines',
    name='SSP5-8.5 (High Emission)',
    line=dict(color='#A23B72', width=3),
    hovertemplate='<b>SSP5-8.5</b><br>' +
                  'Year: %{x}<br>' +
                  'Precipitation: %{y:.2f} mm/year<extra></extra>'
))

# Add ribbon/fill between the lines to show divergence
fig.add_trace(go.Scatter(
    x=avg_precip['year'].tolist() + avg_precip['year'].tolist()[::-1],
    y=avg_precip['low_emissions_pr_mm'].tolist() + avg_precip['high_emissions_pr_mm'].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(162, 59, 114, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    showlegend=False,
    name='Divergence'
))

# Update layout
fig.update_layout(
    title={
        'text': 'U.S. Average Precipitation Projection (2025-2100)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    xaxis=dict(
        title='Year',
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        range=[2025, 2100]
    ),
    yaxis=dict(
        title='Average Precipitation (mm/year)',
        titlefont=dict(size=14),
        tickfont=dict(size=12)
    ),
    hovermode='x unified',
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    ),
    plot_bgcolor='white',
    width=1000,
    height=600,
    margin=dict(l=80, r=50, t=80, b=60)
)

# Add grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')

fig.show()
